In [1]:
%cd ../src
%pwd

/home/muchio1124/Work/Study/NLS_detection/src


'/home/muchio1124/Work/Study/NLS_detection/src'

In [2]:
def clean(seqs):
    cleaned_seq = ''
    for i, seq in enumerate(seqs):
        seq = ''.join(seq[0].split())
        if i == 0:
            cleaned_seq += seq
        else:
            cleaned_seq += seq[-1]
    
    return cleaned_seq

In [3]:
def neighbor_seq(seq, label_list, neighbor=10):
    for i in range(len(label_list)):
        if label_list[i] > 0:
            max_index = i

    start_index = max(0, label_list.index(1) - neighbor)
    end_index = min(len(seq), max_index + neighbor)

    return seq[start_index:end_index]

In [4]:
length = 10
separate_len = 1

fasta_dir = '../data/interim/'

In [5]:
import json
from Bio import SeqIO
from dataset import Dataset

with open('../references/motif_data.json', 'rb') as f:
    motif_data = json.load(f)

陽性の配列周辺を表示

In [6]:
for content in motif_data:
    virus = content['virus'].replace(' ', '_')
    print(virus)
    print('--------')
    
    dataset = Dataset(
        motifs=content['motifs'],
        length=length,
        separate_len=separate_len
    )
    
    fasta_path = f"../data/interim/{virus}.fasta"
    with open(fasta_path, 'r') as f:
        records = [record for record in SeqIO.parse(f, 'fasta')]

    seqs = []
    for record in records:
        label_list = dataset._annotate(record)
        label_list = [int(label > 0) for label in label_list]
        
        if label_list.count(1) > 0:
            seqs.append(neighbor_seq(record.seq, label_list))
        
    n_samples = len(seqs)

    # 重複を除去
    seqs = list(set(seqs))

    for seq in seqs:
        print(seq)

    print("number of samples: {}".format(n_samples))
    
    print("===========================================")

HIV-1
--------
LKKGLGISYGRKKRKHRRGPPQGSK
QTKGLGIYYGRKKRRQRRSAPPSNK
TTKGLGISYGRKKRRQRRRAHQDSQ
ITKGLGISYGRKKRRQRRRSPQSSE
LKKGLGISYGRKKRRPRRAAPLSSK
LKKGLGISYGRKKRRQRRGPPQGSK
QTKGLGISYGRKKRKQRRRAPPSGE
LKKGLGISYGRKKRKHRRGTPQNRE
QTKGLGIYYGRKKRRQRRSAPPSSK
IKKGLGISYGRKKRRQRRRPPPSSP
TTKGLGISYGRKKRRQRRRAHQGSE
TKKGLGISYGRKKRRQRHRTPQGSQ
ITKGLGISYGRKKRRQRRRSPRHSP
ITKGLGISYGRKKRRQRRKPPNGDQ
ITKGLGIYYGRKKRRQRRRAPQDNK
LKKGLGISYGRKKRKHRRGTHPSSQ
QKKGLGISYGRKKRRQRRRAPPSSE
ISKGLGISYGRKKRRQRRRPPQDNQ
QKKGLGISYGRKKRSQRRRTPPSSE
TSKGLGISYGRKKRRRPAASHPDHK
TTKGLGISYGRKKRRQRRRAPQDSK
QTKGLGISYGRKKRRRRRSAPANSE
ATKGLGISYGRKKRRQRHRPHQSSP
LKKGLGISYGRKKRRQRRGTPASLQ
LTKGLGISYGRKKRRRQRHSSASPS
TTKGLGISYGRKKRRQRRRAPQSGP
SKKGLGISYGRKKRRQRRRSSQRSE
LRKGLGIFYGRKKRRQRRRDPQSSQ
LNKGLGISYGRKKRKRRRGTPQSRQ
TTKGLGISYGRKKRRQRRRAHQNSS
TTKGLGISYGRKKRRQRRRTPPDSQ
TTKGLGIYYGRKKRRRRRRAPQGNQ
LKKGLGISYGRKKRRHRRGTPPNRQ
ISKGLGISYGRKKRKQRRRAHQNSE
IQKGLGIFYGRKKRRQRRKPPPSGE
HKKGLRISYGKKKRRQRQRAPQSSE
LKKGLGISYGRKKRKPRRRTPHSSK
LRKGLGIFHGRKKRRRRRGTPHS

replacement_toleranceを1増やして実行し，差分を表示

In [7]:
import copy

with open('../references/motif_data.json', 'rb') as f:
    motif_data = json.load(f)

for content in motif_data:
    virus = content['virus'].replace(' ', '_')
    print(virus)
    print('--------')
    
    dataset = Dataset(
        motifs=content['motifs'],
        length=length,
        separate_len=separate_len)
    
    dataset_rt_up = Dataset(
        motifs=copy.deepcopy(content['motifs']),
        length=length,
        separate_len=separate_len)

    for i in range(len(dataset_rt_up.motifs)):
        dataset_rt_up.motifs[i]['replacement_tolerance'] += 1
        
    fasta_path = f"../data/interim/{virus}.fasta"
    with open(fasta_path, 'r') as f:
        records = [record for record in SeqIO.parse(f, 'fasta')]

    for record in records:
        label_list = dataset._annotate(record)
        label_list = [int(label > 0) for label in label_list]
        label_list_rt_up = dataset_rt_up._annotate(record)
        label_list_rt_up = [int(label > 0) for label in label_list_rt_up]
        
        if (label_list.count(1) == 0) and (label_list_rt_up.count(1) > 0):
            print(neighbor_seq(record.seq, label_list_rt_up))
            
    print("===========================================")

HIV-1
--------
LKKGLGIFHGRKQRGKRRTAPYNSK
Human_adenovirus_5
--------
Simian_virus_40
--------
Equine_infectious_anemia_virus
--------
